# Azure ML Local Run
In this notebook, we create an Azure ML workspace, and use it to locally run the training script.
## Imports and definitions

In [3]:
import os
import shutil
import json
from azureml.core import Workspace, Experiment, ScriptRunConfig
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, BatchAiCompute
from azureml.core.compute_target import ComputeTargetException
import azureml.core
print('azureml.core.VERSION={}'.format(azureml.core.VERSION))

azureml.core.VERSION=0.1.74


## Read in the Azure ML workspace
Read in the the workspace created in a previous notebook.

In [6]:
ws = Workspace.from_config()

Found the config file in: /home/mabou/Source/Repos/MLBatchAIHyperparameterTuning/aml_config/config.json


## Configure a Batch AI cluster
Define the properties of the cluster.

In [11]:
batchai_cluster_name = 'mabouhype'
provisioning_config = BatchAiCompute.provisioning_configuration(
        vm_size='Standard_D4_v2',
        cluster_min_nodes=0,
        cluster_max_nodes=16,
        autoscale_enabled=True)

Create a configured Batch AI cluster, if it doesn't already exist.

In [37]:
if batchai_cluster_name in ws.compute_targets:
    compute_target = ws.compute_targets[batchai_cluster_name]
    if type(compute_target) is not BatchAiCompute:
        raise Exception('Compute target {} is not a Batch AI cluster.'
                        .format(batchai_cluster_name))
else:
    # Create the cluster
    compute_target = ComputeTarget.create(ws, batchai_cluster_name, provisioning_config)

    # You can poll for a minimum number of nodes and set a specific timeout. 
    # If min node count is provided, priovisioning will use the scale settings for the cluster.
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Creating
succeeded
BatchAI wait for completion finished
Minimum number of nodes requested have been provisioned


Print a detailed view of BatchAI cluster status.    

In [39]:
print(compute_target.get_status().serialize())

## Upload the data to the cloud

In [40]:
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='data', overwrite=True, show_progress=True)

$AZUREML_DATAREFERENCE_627f2f4683c248d3b5ee0b8c0c8edd00

Create a system-managed run configuration. Note that `azureml-sdk` is included in the `pip_packages` because it is used by our script.

In [7]:
run_config = RunConfiguration()
run_config.environment.python.user_managed_dependencies = False
run_config.auto_prepare_environment = True
run_config.environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas==0.23.4',
                    'scikit-learn==0.20.0'],
    pip_packages=['azureml-sdk[notebooks, automl]',
                  'lightgbm==2.1.2'])

Specify the script to be run on your local machine. The path to the `data` directory must be absolute.

In [8]:
src = ScriptRunConfig(source_directory=os.path.join('.', 'scripts'), 
                      script='TrainTestClassifier.py', 
                      arguments=['--inputs', os.path.abspath('data'),
                                 '--estimators', '1000',
                                 '--match', '5',
                                 '--ngrams', '2',
                                 '--min_child_samples', '10'],
                      run_config=run_config)

Get an experiment to run the script; create it if it doesn't already exist.

In [9]:
exp = Experiment(workspace=ws, name='mabouhypelocal')

Submit the script to be run. This should return almost immediately.

In [10]:
exp.submit(src)

Experiment,Id,Type,Status,Details Page,Docs Page
mabouhypelocal,mabouhypelocal_1541873329_f8f0e82a,azureml.scriptrun,Running,Link to Azure Portal,Link to Documentation


The experiment returns a table with a link to the `Details Page` in the Azure Portal. That page will let you monitor the status of this run of the experiment, and that of previous runs of that experiment. By clicking on a particular run, you can see its details, files output by the script, and the logs of the run, including the `driver.log` with the script's print outs.

Get an object associated with the latest run. Using this object, you can programmatically control the job. This object was the value returned by the `exp.submit(src)` call.

In [18]:
run_local = list(exp.get_runs())[0]

Wait for the run to complete. This returns a `dict` with detailed information about the run. Here, we see that the run has `Completed`. Other states include `Running` and `Failed`.

In [25]:
run_status = run_local.wait_for_completion()
run_status['status']

'Completed'

We can also get the metrics logged by the script during its execution.

In [27]:
run_local.get_metrics()

{'Accuracy': 0.6544071020925808}

In [11]:
help(exp)

Help on Experiment in module azureml.core.experiment object:

class Experiment(azureml._logging.chained_identity.ChainedIdentity, azureml.core._portal.HasExperimentPortal)
 |  The main entry point into Azure ML Experimentation
 |  
 |  .. remarks::
 |  
 |      An AzureML *experiment* represent the collection of *trials* used to validate a user's *hypothesis*.
 |  
 |      In AzureML, an experiment is represented by the :class:`azureml.core.Experiment` class and a trial is
 |      represented by the :class:`azureml.core.Run` class.
 |  
 |      To get or create an experiment from a workspace, you request the experiment using the experimenet name
 |  
 |      .. code-block:: python
 |  
 |          experiment = Experiment(workspace, "MyExperiment")
 |  
 |      If the experiment is not found in the workspace, a new experiment is created when an experiment trial
 |      is executed.
 |  
 |      There are two ways to execute an experiment trial.  If you are interactively experimenting in

In [28]:
help(run_local)

Help on ScriptRun in module azureml.core.script_run object:

class ScriptRun(azureml.core.run.Run)
 |  An experiment run class.
 |  :param experiment: The experiment object
 |  :type experiment: azureml.core.experiment.Experiment
 |  :param run_id: Run id.
 |  :type run_id: str
 |  :param directory: The source directory
 |  :type directory: str
 |  :param _run_config:
 |  :type _run_config: RunConfiguration
 |  :param kwargs:
 |  :type kwargs: dict
 |  
 |  Method resolution order:
 |      ScriptRun
 |      azureml.core.run.Run
 |      azureml._run_impl.run_base._RunBase
 |      azureml._logging.chained_identity.ChainedIdentity
 |      azureml.core._portal.HasRunPortal
 |      azureml.core._portal.HasExperimentPortal
 |      azureml.core._portal.HasWorkspacePortal
 |      azureml.core._portal.HasPortal
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, experiment, run_id, directory=None, _run_config=None, **kwargs)
 |      The constructor method
 |  
 |  can

In [4]:
help(ComputeTarget)

Help on class ComputeTarget in module azureml.core.compute.compute:

class ComputeTarget(abc.ABC)
 |  Abstract parent class for all compute targets that are managed by AzureML. ComputeTarget constructor is used to
 |  retrieve a cloud representation of a Compute object associated with the provided workspace. Will return an
 |  instance of a child class corresponding to the specific type of the retrieved Compute object.
 |  
 |  :param workspace: The workspace object containing the Compute object to retrieve
 |  :type workspace: azureml.core.workspace.Workspace
 |  :param name: The name of the of the Compute object to retrieve
 |  :type name: str
 |  
 |  Method resolution order:
 |      ComputeTarget
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, workspace, name)
 |      ComputeTarget constructor is used to retrieve a cloud representation of a Compute object associated with the
 |      provided workspace. Will return an instance of a chil